# Face Verification and Recognition 

background knowledge (original source from Coursera):

**Face Verification** "Is this the claimed person?" For example, at some airports, you can pass through customs by letting a system scan your passport and then verifying that you (the person carrying the passport) are the correct person. A mobile phone that unlocks using your face is also using face verification. This is a 1:1 matching problem.

**Face Recognition** "Who is this person?" For example, the video lecture showed a [face recognition video](https://www.youtube.com/watch?v=wr4rx0Spihs) of Baidu employees entering the office without needing to otherwise identify themselves. This is a 1:K matching problem.

[FaceNet](https://arxiv.org/pdf/1503.03832.pdf)

[DeepFace](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf).

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import PIL

%matplotlib inline
%load_ext autoreload
%autoreload 2

### Using a ConvNet to Compute Encodings

In [2]:
json_file = open('keras-facenet-h5/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('keras-facenet-h5/model.h5')

### (a) Face Verification

In [3]:
def img_to_encoding(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
    img = np.around(np.array(img) / 255.0, decimals=12)
    x_train = np.expand_dims(img, axis=0)
    embedding = model.predict_on_batch(x_train)
    return embedding / np.linalg.norm(embedding, ord=2)

In [4]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", model)
database["younes"] = img_to_encoding("images/younes.jpg", model)
database["tian"] = img_to_encoding("images/tian.jpg", model)
database["andrew"] = img_to_encoding("images/andrew.jpg", model)
database["kian"] = img_to_encoding("images/kian.jpg", model)
database["dan"] = img_to_encoding("images/dan.jpg", model)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", model)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", model)
database["kevin"] = img_to_encoding("images/kevin.jpg", model)
database["felix"] = img_to_encoding("images/felix.jpg", model)
database["benoit"] = img_to_encoding("images/benoit.jpg", model)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", model)

In [5]:
def verify(image_path, identity, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".

    """
    encoding = img_to_encoding(image_path, model)
    dist = np.linalg.norm((encoding-database[identity]))

    if dist<0.7:
        print("It's " + str(identity) + ", welcome in!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False

    return dist, door_open

In [6]:
distance, door_open_flag = verify("images/camera_0.jpg", "younes", database, model)
assert np.isclose(distance, 0.5992949), "Distance not as expected"

It's younes, welcome in!


### (b) Face Recognition

In [10]:
def who_is_it(image_path, database, model):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.
    
    """
    encoding =  img_to_encoding(image_path, model)
    min_dist = 100
    
    for (name, db_enc) in database.items():
        
        dist = np.linalg.norm(encoding - db_enc)

        if dist < min_dist:
            min_dist = dist
            identity = name

    if min_dist > 0.8:
        print("Not in the database.")
    else:
        print ("it's " + str(identity))
        
    return min_dist, identity

In [11]:
test1 = who_is_it("images/camera_0.jpg", database, model)

it's younes
